In [26]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely.geometry
import shapely
import rasterio
import rioxarray
from rasterio.features import shapes
from shapely.geometry import shape
import matplotlib.pyplot as plt
from pathlib import Path

In [27]:
mwperkm2_wind = 4.5 ## originally 9 MW/km2 but deduct by 50% of the technically available from IEA's Thailand CET
mwperkm2_solar = 15 ## originally 30 MW/km2 but deduct by 50% of the technically available from IEA's Thailand CET

lccs_resoultion = (300/1000) ** 2 ## meters
## max capacity of spp 90 MW
maxcapacityinpolygon = 90

maxnoofgrid_solar = round((maxcapacityinpolygon/(mwperkm2_solar * (lccs_resoultion**2))),0) + 9
maxnoofgrid_wind = round((maxcapacityinpolygon/(mwperkm2_wind * (lccs_resoultion**2))),0) + 9

In [28]:
mask = None
with rasterio.Env():
    with rasterio.open('Output_SolarWind\\xr_AVA_Solar.nc') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform,connectivity=8)))


geoms = list(results)

In [29]:
print(len(geoms))

23781


In [30]:
gpd_polygonized_raster_solar  = gpd.GeoDataFrame.from_features(geoms)
gpd_polygonized_raster_solar.crs = {'init': 'epsg:4326'}
gpd_polygonized_raster_solar= gpd_polygonized_raster_solar.to_crs({'init': 'epsg:3857'})
gpd_polygonized_raster_solar['area'] = gpd_polygonized_raster_solar['geometry'].area / (10**6)
gpd_polygonized_raster_solar = gpd_polygonized_raster_solar.loc[gpd_polygonized_raster_solar['raster_val'] != 0]
print(len(gpd_polygonized_raster_solar.loc[gpd_polygonized_raster_solar['area'] < 0.4]))

C:\Users\basna\AppData\Roaming\Python\Python311\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\basna\AppData\Roaming\Python\Python311\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


6742


In [31]:
print(gpd_polygonized_raster_solar['area'].mean())
print(gpd_polygonized_raster_solar['area'].max())


4.728085502987531
1755.926127605849


In [32]:
df_final_SI_Wind = pd.read_csv('Output_SolarWind\\df_final_SI_Solar.csv')

In [33]:
print(df_final_SI_Wind['area'].mean())
print(df_final_SI_Wind['area'].max())

6.468036598139861
1755.727782529546
